# An Inefficient Vector Space Model

In [1]:
from collections import defaultdict
from math import log, sqrt
import re

The dataset is the TIME dataset, available at http://ir.dcs.gla.ac.uk/resources/test_collections/time/

In [2]:
def import_dataset():
    """
    This function import all the articles in the TIME corpus,
    returning list of lists where each sub-list contains all the
    terms present in the document as a string.
    """
    articles = []
    with open('TIME.ALL', 'r') as f:
        tmp = []
        for row in f:
            if row.startswith("*TEXT"):
                if tmp != []:
                    articles.append(tmp)
                tmp = []
            else:
                row = re.sub(r'[^a-zA-Z\s]+', '', row)
                tmp += row.split()
    return articles

In [3]:
def make_inverted_index(articles):
    """
    This function builds an inverted index as an hash table (dictionary)
    where the keys are the terms and the values are ordered lists of
    docIDs containing the term.
    """
    index = defaultdict(set)
    for docid, article in enumerate(articles):
        for term in article:
            index[term].add(docid)
    return index

In [4]:
def make_positional_index(articles):
    """
    A more advanced version of make_inverted_index. Here each posting is
    non only a document id, but a list of positions where the term is
    contained in the article.
    """
    index = defaultdict(dict)
    for docid, article in enumerate(articles):
        for pos, term in enumerate(article):
            try:
                index[term][docid].append(pos)
            except KeyError:
                index[term][docid] = [pos]
    return index

In [13]:
def documents_as_vectors(articles):
    """
    Here we generate a list of dictionaries. Each element of the list
    represents a document and each document has an associated dict where
    to each term the corresponding tf-idf is associated. Since this function
    creates a structure of size O(#documents \times #terms), it can
    be used only for small collections.
    """
    p_index = make_positional_index(articles)
    vectors = []
    n = len(articles)
    idf = {}
    for term in p_index.keys():
        idf[term] = log(n/len(p_index[term]))
    for docid in range(0, len(articles)):
        # We create a dictionary with a key for each dimension (term)
        v = {}
        for term in p_index.keys():
            try:
                tfidf = len(p_index[term][docid]) * idf[term]
            except KeyError:
                tfidf = 0
            v[term] = tfidf
        vectors.append(v)
    return vectors

In [14]:
def show_document_vector(v, docid):
    """
    This function prints, for a document represented as a vector in v, all the
    non-zero weights (both normalized and not) and the corresponding terms
    """
    non_zero_terms = [x for x in v[docid].keys() if v[docid][x] > 0]
    vector = [(x, v[docid][x]) for x in non_zero_terms]
    vector.sort(key=lambda x: x[1], reverse=True)
    length = sqrt(sum([x[1]**2 for x in vector]))
    normalized = {k: tfidf/length for k, tfidf in vector}
    for (term, tfidf) in vector:
        print(f"{term}:\t{tfidf}\t(normalized: {normalized[term]})")

In [17]:
# Example of usage
articles = import_dataset()
vectors = documents_as_vectors(articles)

In [24]:
" ".join(articles[2])

'BERLIN ONE LAST RUN HANS WEIDNER HAD BEEN HOPING FOR MONTHS TO ESCAPE DRAB EAST GERMANY AND MAKE HIS WAY TO THE WEST THE ODDS WERE AGAINST HIM FOR WEIDNER WAS A CRIPPLE ON CRUTCHES WHO LIVED IN THE VILLAGE OF NEUGERSDORF MILES SOUTHEAST OF THE FRONTIER OF FREEDOM BUT HANS WEIDNER DID HAVE ONE MAJOR ASSET THE BUS THAT HE OPERATED FOR THE LOCAL COMMUNIST REGIME IT WAS AN UGLY THING AND ANCIENT ITS CHASSIS CREAKED AND THE ENGINE COUGHED A CREAMCOLORED COAT OF PAINT COULD NOT DISGUISE THE WELTS AND BRUISES OF TWO DECADES OF CHUGGING SERVICE IN FACT THE BUS WAS READY FOR THE JUNK PILE WHEN WEIDNER DECIDED TO PRESS IT INTO SERVICE FOR ONE LAST RUN SHARP BLADES THE HAZARDS WOULD BE GREAT ON THE JOURNEY TO THE BORDER SO WEIDNER SIGNED UP A FELLOW VILLAGER JURGEN WAGNER TO TAKE THE WHEEL EIGHT DAYS BEFORE CHRISTMAS THE PAIR BEGAN THE FEVERISH PREPARATIONS IN WEIDNERS GARAGE FIRST WEIDNER AND WAGNER ATTACHED A HEAVY SNOWPLOW TO THE FRONT OF THE BUS NOT TO PLOW SNOW BUT TO SCOOP AWAY THE HEAVY O

In [25]:
show_document_vector(vectors, 2)

WEIDNER:	48.360042512288096	(normalized: 0.534471813494657)
BUS:	25.52977028866349	(normalized: 0.2821532388193592)
WAGNER:	24.180021256144048	(normalized: 0.2672359067473285)
POTATOES:	13.306702204805735	(normalized: 0.14706474373401815)
WAHAH:	12.090010628072024	(normalized: 0.13361795337366425)
BERLIN:	11.815851442710784	(normalized: 0.1305879651980127)
BLADES:	10.703716266952133	(normalized: 0.1182967248814251)
AUTOBAHN:	9.892786050735804	(normalized: 0.10933438074848466)
HANS:	8.871134803203823	(normalized: 0.09804316248934543)
WHEEL:	8.871134803203823	(normalized: 0.09804316248934543)
CHECKPOINT:	8.871134803203823	(normalized: 0.09804316248934543)
CHRISTMAS:	7.931127544712352	(normalized: 0.0876542678969468)
STOP:	7.833054328652597	(normalized: 0.08657036956022819)
YARDS:	7.695561473399585	(normalized: 0.08505080812330507)
STEEL:	6.811895968841506	(normalized: 0.07528459866176831)
SHARP:	6.673910225867603	(normalized: 0.07375958986416584)
WIVES:	6.544833183592461	(normalized: 0.0